# Monitor（Todo）

本节将综合介绍 Autogen 中用于监控 MAS 运行过程的各种方法。包含有：

- Logging 



参考：

1. https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/logging.html



## Logging

AutoGen 使用的是 Python 内置的 `logging` 模块。

Python 的 logging 模块是标准日志库，支持：

一、 多个日志等级：控制“输出什么级别的日志”

1. DEBUG: 最详细，适合开发时看程序内部状态

2. INFO: 正常流程的消息

3. WARNING: 警告信息，程序还能继续运行

4. ERROR: 错误信息，某些操作失败了

5. CRITICAL: 严重错误，程序可能崩溃

二、多个日志对象（logger）：每个对象负责一类日志，比如“系统日志”、“模型调用日志”、“监控日志”等。

每个 logger 都可以单独设置日志等级和输出方式（比如写入文件、打印到屏幕）。
Python 的 logging 模块可以输出各种层级的日志：DEBUG、INFO、WARNING、ERROR、CRITICAL。

AutoGen 中主要使用了两个日志名称（logger name）：

| Logger 名称                                       | 含义         | 适用场景                     |
| ----------------------------------------------- | ---------- | ------------------------ |
| `autogen.agentchat.trace` (`TRACE_LOGGER_NAME`) | 记录详细追踪信息   | 如：Agent之间发送消息的详细过程       |
| `autogen.agentchat.event` (`EVENT_LOGGER_NAME`) | 记录结构化的事件信息 | 如：一个Agent发出消息、接收回复、调用工具等 |




要为 AgentChat 启用日志记录功能，你可以使用以下代码：

In [1]:
import os
import asyncio
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
import logging
from autogen_agentchat import EVENT_LOGGER_NAME, TRACE_LOGGER_NAME

# 设置全局日志等级（默认最低等级为 WARNING）
logging.basicConfig(level=logging.WARNING)

# 启用详细执行追踪日志（trace）"
trace_logger = logging.getLogger(TRACE_LOGGER_NAME)
trace_logger.addHandler(logging.StreamHandler())  # 输出到终端
trace_logger.setLevel(logging.DEBUG)              # 设置为 DEBUG 级别

# 启用结构化事件日志（event），记录智能体间低层消息
event_logger = logging.getLogger(EVENT_LOGGER_NAME) #获取logger
event_logger.addHandler(logging.StreamHandler())  # 输出到终端
event_logger.setLevel(logging.DEBUG)              # 设置为 DEBUG 级别

handler = logging.FileHandler("./resources/log/autogen_debug.log")
# 将日志写入这个文件（而不仅限于控制台），最终效果：日志同时输出到控制台和文件，便于后续查看、调试、分析
trace_logger.addHandler(handler)
event_logger.addHandler(handler)
load_dotenv()
siliconflow_api_key = os.getenv("SILICONFLOW_API_KEY") # 读取你的 OPENAI API key

# 初始化 OpenAIChatCompletionClient 客户端，连接到硅基流动平台的 Qwen3-8B 模型
model_client = OpenAIChatCompletionClient(
    model="Qwen/Qwen3-14B",                # 指定要调用的模型名称，硅基流动平台上 Qwen 3-8B 模型
    base_url="https://api.siliconflow.cn/v1",  # 硅基流动平台的 API 访问地址
    api_key=siliconflow_api_key,  # 你的 API 密钥
    model_info={                        
        "family": "qwen",              
        "context_length": 8192,        
        "max_output_tokens": 2048,     
        "tool_choice_supported": True, 
        "tool_choice_required": False,  
        "structured_output": True,     
        "vision": False,                
        "function_calling": True,      
        "json_output": True,
        "multiple_system_messages":True
    },
)

In [2]:
assistant = AssistantAgent(
    name="assistant",
    model_client=model_client,
    system_message="你是一个乐于助人的中文助手。"
)

user_proxy = UserProxyAgent(name="user", input_func=input)

# 创建终止条件：当用户回复 "APPROVE" 时结束对话
termination = TextMentionTermination("end")

# 创建团队
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# # 运行团队对话，并将消息流输出到控制台
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# # 在脚本中建议用 asyncio.run(...) 启动
await Console(stream)

---------- TextMessage (user) ----------
Write a 4-line poem about the ocean.
---------- ThoughtEvent (assistant) ----------

Okay, the user asked for a 4-line poem about the ocean. Let me start by thinking about the main elements of the ocean that are poetic. Waves, tides, maybe the vastness or the creatures living in it.

They might want imagery that evokes emotions, like calmness or power. I should consider the structure—four lines, maybe a rhyme scheme. Let's see, ABAB or AABB? Maybe AABB for simplicity and a more rhythmic feel.

First line: Something about the ocean's movement. "Waves whisper secrets to the shore" – that personifies the ocean and creates a sense of mystery.

Second line: Needs to rhyme with "shore." Maybe "Tides dance in rhythms, soft and sure." That adds motion and a gentle, consistent action.

Third line: Transition to something deeper, like the ocean's depths. "Endless depths where mysteries are stored" – emphasizes the unknown and the vastness.

Fourth line: R

Enter your response:  ok


---------- TextMessage (user) ----------
ok
---------- ThoughtEvent (assistant) ----------

好的，用户让我写一首关于海洋的四行诗，然后他们回复了“ok”。现在我需要分析用户的意图和需求，然后给出合适的回应。

首先，用户之前请求了一首四行诗，我给出了一个中文的版本，包含了潮汐、深蓝梦境、鲸歌和月光这些意象。用户现在回复“ok”，可能是在确认收到了这首诗，或者表示满意，也可能只是简单的回应，没有进一步的意思。

接下来，我需要判断用户是否需要更多的帮助。他们可能想继续讨论诗歌，或者需要修改，或者有其他相关的问题。不过“ok”本身比较中性，可能只是确认，也可能在等待下一步的行动。

考虑到之前的对话中用户明确要求了一首诗，并且得到了回应，现在他们可能希望得到反馈，或者想了解诗的创作思路。也有可能用户想继续用中文创作，或者转而用英文。不过用户并没有明确说明，所以需要进一步询问。

另外，用户可能对我的诗有疑问，或者想看看不同的风格。例如，是否有什么特别的押韵或结构，或者是否希望加入更多的自然元素。也有可能他们想知道创作灵感来源，或者是否有其他主题的诗可以参考。

我应该保持友好和开放的态度，邀请用户提出更多问题或继续讨论。比如询问是否满意，是否有需要调整的地方，或者是否需要其他形式的帮助。同时，保持鼓励用户深入交流，避免他们感到被忽视。

需要注意的是，用户可能没有足够的中文诗歌知识，可能需要解释诗中的意象或用词。或者，他们可能希望这首诗有更具体的主题，比如海洋的神秘、美丽，或者与人类的关系。

也有可能用户是在测试我的反应，所以需要确保回应既专业又亲切，提供进一步协助的机会。避免过于机械，保持自然。

总结，用户现在可能需要确认诗是否符合预期，或者希望得到解释，或者有其他需求。因此，我的回应应该是开放式的，邀请用户进一步交流，并且保持帮助的态度。

---------- TextMessage (assistant) ----------


You're welcome! If you'd like to explore more poetic ideas or need help refining this one, I'm happy to assist. Would yo

Enter your response:  end


---------- TextMessage (user) ----------
end


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a 4-line poem about the ocean.', type='TextMessage'), ThoughtEvent(source='assistant', models_usage=None, metadata={}, content='\nOkay, the user asked for a 4-line poem about the ocean. Let me start by thinking about the main elements of the ocean that are poetic. Waves, tides, maybe the vastness or the creatures living in it.\n\nThey might want imagery that evokes emotions, like calmness or power. I should consider the structure—four lines, maybe a rhyme scheme. Let\'s see, ABAB or AABB? Maybe AABB for simplicity and a more rhythmic feel.\n\nFirst line: Something about the ocean\'s movement. "Waves whisper secrets to the shore" – that personifies the ocean and creates a sense of mystery.\n\nSecond line: Needs to rhyme with "shore." Maybe "Tides dance in rhythms, soft and sure." That adds motion and a gentle, consistent action.\n\nThird line: Transition to something deeper, like the ocean\'s 